In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from model import DeepHit
from losses import *
from utils import *
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
tf.debugging.set_log_device_placement(True)
tf.keras.backend.set_floatx('float64')

In [3]:
df = pd.read_csv("./data/metabric.csv")

In [4]:
event = np.asarray(df[['label']])
time = np.asarray(df[['event_time']])
dhdata = np.asarray(df.iloc[:,:-2])

In [5]:
scaler = StandardScaler()
df_trans = scaler.fit_transform(dhdata)

In [6]:
num_category = int(np.max(time) * 1.2)        #to have enough time-horizon
num_event = int(len(np.unique(event)) - 1) #only count the number of events (do not count censoring as an event)
x_dim = df_trans.shape[1]
mask1 = compute_mask1(time, event, num_event, num_category)
mask2 = compute_mask2(time, -1, num_category)

In [7]:
parameters = get_random_hyperparameters()

In [8]:
deephit = DeepHit(num_layers_shared=parameters['num_layers_shared'], h_dim_shared=parameters['h_dim_shared'],
                  activation=parameters['active_fn'], dropout_rate=parameters['dropout'],
                num_layers_cs=parameters['num_layers_CS'], h_dim_cs=parameters['h_dim_CS']
                  , num_event=num_event, num_category=num_category)
optimizer = tf.keras.optimizers.Adam(parameters['lr_train'])

In [9]:
def run_optimization(x, event, time, mask1, mask2, alpha, beta, gamma):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = deephit(x)
        loss1 = loss_log_likelihood(pred, mask1, event)
        loss2 = loss_ranking(pred, mask2, time, event, num_event, num_category)
        loss3 = loss_calibration(pred, mask2, time, event, num_event, num_category)
        total_loss = alpha * loss1 + beta * loss2 + gamma * loss3
        # Compute gradients.
        gradients = g.gradient(total_loss, deephit.trainable_variables)

        # Update W and b following gradients.
        optimizer.apply_gradients(zip(gradients, deephit.trainable_variables))

In [10]:
(tr_data,te_data, tr_event,te_event, tr_time,te_time, 
     tr_mask1,te_mask1, tr_mask2,te_mask2)  = train_test_split(df_trans, event, time, mask1, mask2,
                                                               test_size=0.20, random_state=2020)

In [11]:
# Use tf.data API to shuffle and batch data.
train_data = tf.data.Dataset.from_tensor_slices((tr_data, tr_event, tr_time, tr_mask1, tr_mask2))
train_data = train_data.repeat().shuffle(5000).batch(parameters["mb_size"]).prefetch(1)

Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousRandomSeedGenerator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ShuffleDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op BatchDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0


In [12]:
mirrored_strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [13]:
# Run training for the given number of steps.
for step, (batch_x, batch_event, batch_time, batch_mask1, batch_mask2) in enumerate(train_data.take(parameters['iteration']), 1):
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_event, batch_time, batch_mask1, batch_mask2,
                    parameters["alpha"], parameters["beta"], parameters["gamma"])
    display_step = 100
    if step % display_step == 0:
        pred = deephit(batch_x)
        loss1 = loss_log_likelihood(pred, batch_mask1, batch_event)
        loss2 = loss_ranking(pred, batch_mask2, batch_time, batch_event, num_event, num_category)
        loss3 = loss_calibration(pred, batch_mask2, batch_time, batch_event, num_event, num_category)
        total_loss = parameters["alpha"] * loss1 + parameters["beta"] * loss2 + parameters["gamma"] * loss3
        print("step: %i, loss: %f" % (step, total_loss))

Executing op TakeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /jo

KeyboardInterrupt: 